In [14]:
import numpy as np

In [207]:
PLAYER_B = 1
PLAYER_W = -PLAYER_B
CELL_EMPTY = 0
DIRNS = np.array([(xs, ys) for xs in [-1, 0, 1] for ys in [-1, 0, 1] if xs or ys])

def shift_indices(shift, side):
    """get indices for shifting array in one dimension"""
    if shift == 0:
        orig_idx = range(side)
        shifted_idx = range(side)
    elif shift > 0:
        orig_idx = range(side - shift)
        shifted_idx = range(shift, side)
    elif shift < 0:
        orig_idx = range(-shift, side)
        shifted_idx = range(side + shift)
    else:
        orig_idx, shifted_idx = None, None
    return orig_idx, shifted_idx

class Game(object):
    side = 8
    
    def __init__(self):
        """Reset board to starting position"""
        self.reset()
        self.turn = PLAYER_B # whos turn is next
    
    def reset(self):
        """Reset board to starting position"""
        self.board = np.zeros((self.side, self.side), dtype=np.int8)
        # 1 = black, -1 = white
        self.board[4, 3] = 1
        self.board[3, 4] = 1
        self.board[3, 3] = -1
        self.board[4, 4] = -1
    
    def _neighbour_candidate_cells(self):
        """List neighbours that would make turn candidates
        a cell is a good candidate if (a) it is empty and (b) any neighbour is the opposite of self.turn
        """
        empty_cells = np.zeros((self.side, self.side), dtype=np.int8)
        empty_cells[self.board == CELL_EMPTY] = 1
        # suitable neighbours with opponents stones
        opponent_cells = np.zeros((self.side, self.side), dtype=np.int8)
        opponent_cells[self.board == self.opponent] = 1
        # 8 possible directions to a neighbour
        shifts = DIRNS
        # an 8x8x9 array of conditions 
        neighbour_array = np.zeros((len(shifts), self.side, self.side), dtype=np.int8)
        for idx, (rs, cs) in enumerate(shifts):
            ridx_orig, ridx_shifted = shift_indices(rs, self.side)
            cidx_orig, cidx_shifted = shift_indices(cs, self.side)
            neighbour_array[np.ix_([idx], ridx_shifted, cidx_shifted)] = opponent_cells[np.ix_(ridx_orig, cidx_orig)]
        # cells that have a neighbour of opponent's colour
        cells_with_opponent_neighbours = np.any(neighbour_array, axis=0)
        candidates = np.logical_and(empty_cells, cells_with_opponent_neighbours)
        return candidates
    
    def _attemp_turn(self, r, c):
        """Attempt to place the next piece at (r, c)
        Turn is decided based on self.turn
        """
        # if we are trying to place in a non-empty cell, return
        if self.board[r, c] != CELL_EMPTY:
            return False   
        
        # else, try to turn along each diagonal
        turned_idx = []
        for (rs, cs) in DIRNS:
            rr, cc = r + rs, c + cs
            diag_turned_idx = []
            diag_terminated = False
            # proceed until we hit an edge, or an empty cell, or our own cell
            while 0 <= rr and rr < self.side and 0 <= cc and cc < self.side:
                if self.board[rr, cc] == self.opponent: # continue along unbroken line of opponent's stones
                    diag_turned_idx += [(rr, cc)]
                    rr, cc = rr + rs, cc + cs
                elif self.board[rr, cc] == self.player: # terminate at our own stone
                    diag_terminated = True
                    break
                elif self.board[rr, cc] == CELL_EMPTY: # terminate (unsuccesfully) at an empty cell
                    break
            # if diagonal completed successfully, append cells to be turned
            if diag_terminated:
                turned_idx += diag_turned_idx
        
        # if we have turned stones along any of the diagonals
        if turned_idx:
            ridx, cidx = zip(*turned_idx)
            self.board[ridx, cidx] = self.player
            self.board[r, c] = self.player
            self.turn = self.opponent
            return True
        else:
            return False
    
    @property
    def player(self):
        """current player"""
        return self.turn
    
    @property
    def opponent(self):
        """opponent player"""
        return -self.turn
    
    @property
    def score(self):
        """Compute totals for each player"""
        counts = dict(zip(*np.unique(self.board, return_counts=True)))
        b, w = counts[1], counts[-1]
        return b, w
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        """Return a string representation of the board"""
        chars = {
            0: ".",
            1: "Ø",  #®
            -1: "O",  #©
        }
        def row_str(row):
            cell_chars = [chars[r] for r in row]
            return ''.join(cell_chars)
        row_strings = [row_str(row) for row in self.board]
        score_string = 'Score: B=%d, W=%d' % self.score
        turn_string = 'Next turn: %s' % ('B' if self.turn == PLAYER_B else 'W')
        return '\n'.join([score_string, turn_string] + row_strings) 
        
    

In [208]:
g = Game()
print g

Score: B=2, W=2
Next turn: B
........
........
........
...OØ...
...ØO...
........
........
........


In [209]:
print g._attemp_turn(2,3)
print g._attemp_turn(2,2)

True
True


In [210]:
g

Score: B=3, W=3
Next turn: B
........
........
..OØ....
...OØ...
...ØO...
........
........
........